<a href="https://colab.research.google.com/github/Anjanisinghthakur/Stratascratch-Solutons/blob/main/Election_Results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#The election is conducted in a city and everyone can vote for one or more candidates, or choose not to vote at all. Each person has 1 vote so if they vote for multiple candidates, their vote gets equally split across these candidates. For example, if a person votes for 2 candidates, these candidates receive an equivalent of 0.5 vote each. Some voters have chosen not to vote, which explains the blank entries in the dataset.


#Find out who got the most votes and won the election. Output the name of the candidate or multiple names in case of a tie.
#To avoid issues with a floating-point error you can round the number of votes received by a candidate to 3 decimal places.

#Table: voting_results

In [ ]:
#Pyspark solution
#Import your libraries
import pyspark
from pyspark.sql.functions import *
from pyspark.sql.window import Window
# Start writing code
voting_results
voting_results=voting_results.filter(voting_results.candidate.isNotNull())
#voting_results.show()
spec1=Window.partitionBy(voting_results.voter)
spec2=Window.partitionBy(voting_results.voter,voting_results.candidate)
res=voting_results.withColumn('total_vpc', count(voting_results.candidate).over(spec1)).withColumn('ind', lit(1))
res=res.withColumn('vote_count', res.ind/res.total_vpc)
res=res.select(res.candidate, res.vote_count)
res=res.groupBy(res.candidate).agg(sum(res.vote_count).alias('vote_count'))

res=res.withColumn('rnk', rank().over(Window.orderBy(res.vote_count.desc())))
res=res.select(res.candidate).where(res.rnk==1)
res.show()

# To validate your solution, convert your final pySpark df to a pandas df
res.toPandas()

In [ ]:
#SQL solution
with temp as(
select voter, candidate, count(voter) over(partition by voter) count_each,1 as new
from voting_results
where candidate != ''
)
select candidate from
(select candidate, sum(prob) prob from(
select candidate,(new/count_each) prob  from temp
)
a
group by 1
order by 2 desc
)b
limit 1